In [1]:
import geopandas as gpd
import pandas as pd
import json
import numpy as np
import altair as alt
import datetime

In [2]:
covid = pd.read_csv('covid.csv')
covid_us = pd.read_csv('covid_us.csv')
covid = covid.rename({'Country/Region': 'Country'}, axis=1)
vaccine = pd.read_csv('country_vaccinations.csv')
vaccine

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.00,0.00,NaN,NaN,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm...",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm...",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm...",World Health Organization,https://covid19.who.int/
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm...",World Health Organization,https://covid19.who.int/
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm...",World Health Organization,https://covid19.who.int/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17602,Zimbabwe,ZWE,2021-05-08,657838.0,509274.0,148564.0,17076.0,19648.0,4.43,3.43,1.00,1322.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1392575941...
17603,Zimbabwe,ZWE,2021-05-09,684243.0,526066.0,158177.0,26405.0,22863.0,4.60,3.54,1.06,1538.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1392575941...
17604,Zimbabwe,ZWE,2021-05-10,690653.0,529360.0,161293.0,6410.0,21877.0,4.65,3.56,1.09,1472.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1392575941...
17605,Zimbabwe,ZWE,2021-05-11,709772.0,539526.0,170246.0,19119.0,21428.0,4.78,3.63,1.15,1442.0,Sinopharm/Beijing,Ministry of Health,https://twitter.com/MoHCCZim/status/1392575941...


In [3]:
data = covid[covid
      .Country
      .isin(['China','India','United Kingdom','Brazil'])].groupby('Country').sum().reset_index().drop(['Lat', 'Long'],axis=1)
df = data.transpose().reset_index()
new_header = df.iloc[0] #grab the first row for the header
df = df[1:]
df.columns = new_header
df = df.rename({'Country': 'date'}, axis=1)
df       

,date,Brazil,China,India,United Kingdom
1,1/22/20,0,548,0,0
2,1/23/20,0,643,0,0
3,1/24/20,0,920,0,0
4,1/25/20,0,1406,0,0
5,1/26/20,0,2075,0,0
...,...,...,...,...,...
466,5/1/21,14725975,102517,19557457,4434157
467,5/2/21,14754910,102530,19925517,4435831
468,5/3/21,14779529,102549,20282833,4437505
469,5/4/21,14856888,102560,20664979,4439489


In [4]:
us_data = covid_us.groupby('Country_Region').sum().reset_index().drop(['Lat', 'Long_', 'UID', 'code3', 'FIPS'],axis=1
                                             ).transpose().reset_index()
new_header = us_data.iloc[0] #grab the first row for the header
us_data = us_data[1:]
us_data.columns = new_header
us_data = us_data.rename({'Country_Region':'date', 'US':'United States'}, axis=1)
us_data

,date,United States
1,1/22/20,1
2,1/23/20,1
3,1/24/20,2
4,1/25/20,2
5,1/26/20,5
...,...,...
466,5/1/21,32392274
467,5/2/21,32421641
468,5/3/21,32472201
469,5/4/21,32512934


In [5]:
df = pd.merge(df, us_data, how='left', on=['date'])

In [6]:
df['date'] = df['date'].apply(lambda x : '20' + x[-2:] + '-' + '0'*(2-x.find('/',1)) + x[0:x.find('/',1)] + '-' + '0'*(x.find('/',1)-x.find('/',3)+3) + x[x.find('/',1)+1:x.find('/',3)])
df

,date,Brazil,China,India,United Kingdom,United States
0,2020-01-22,0,548,0,0,1
1,2020-01-23,0,643,0,0,1
2,2020-01-24,0,920,0,0,2
3,2020-01-25,0,1406,0,0,2
4,2020-01-26,0,2075,0,0,5
...,...,...,...,...,...,...
465,2021-05-01,14725975,102517,19557457,4434157,32392274
466,2021-05-02,14754910,102530,19925517,4435831,32421641
467,2021-05-03,14779529,102549,20282833,4437505,32472201
468,2021-05-04,14856888,102560,20664979,4439489,32512934


In [7]:
cleaned = df.melt('date', var_name='country', value_name='infection_count')
country = pd.DataFrame(cleaned.country.unique()).rename(columns={0:'country'})
cleaned

,date,country,infection_count
0,2020-01-22,Brazil,0
1,2020-01-23,Brazil,0
2,2020-01-24,Brazil,0
3,2020-01-25,Brazil,0
4,2020-01-26,Brazil,0
...,...,...,...
2345,2021-05-01,United States,32392274
2346,2021-05-02,United States,32421641
2347,2021-05-03,United States,32472201
2348,2021-05-04,United States,32512934


In [8]:
vac = vaccine[vaccine
      .country
      .isin(['China','India','United Kingdom','Brazil','United States'])].reset_index()[['date', 'country', 'total_vaccinations']]
df = pd.merge(cleaned, vac, how='left', on=['date','country']).dropna().reset_index().drop(['index'],axis=1)
df

,date,country,infection_count,total_vaccinations
0,2021-01-16,Brazil,8455059,0.0
1,2021-01-17,Brazil,8488099,112.0
2,2021-01-18,Brazil,8511770,1109.0
3,2021-01-19,Brazil,8573864,11470.0
4,2021-01-20,Brazil,8638249,28543.0
...,...,...,...,...
500,2021-05-01,United States,32392274,243463471.0
501,2021-05-02,United States,32421641,245591469.0
502,2021-05-03,United States,32472201,246780203.0
503,2021-05-04,United States,32512934,247769049.0


## We want to explore the relationship between the number of infections and the number of vaccinations. Here, we only look at a few countries which have significant number of vaccines.

In [9]:
drop_down = alt.binding_select(options=list(cleaned.country.unique()))
country_select = alt.selection_single(fields=['country'], bind=drop_down, name="Select")
color = alt.condition(country_select, alt.Color('country:N'), alt.value('lightgray'))

vac_chart = alt.Chart(df, width=400, height=400, title='Time series cumulative vaccine data').mark_line().encode(
    x='date:O',
    y='total_vaccinations:Q',
    color=color,
    tooltip=['date', 'total_vaccinations']
).add_selection(
    country_select
).interactive()
count_chart = alt.Chart(df, width=400, height=400, title='Time series cumulative infection data').mark_line().encode(
    x='date:O',
    y='infection_count:Q',
    color=color,
    tooltip=['date', 'infection_count']
).add_selection(
    country_select
).interactive()
count_chart|vac_chart

alt.HConcatChart(...)

## We want to explore the relationship between the number of vaccinations and the nubmer of infections across different plots, this time particularly choosing some time slots for comparison.

In [10]:
brush = alt.selection_interval()

base1 = alt.Chart(df).mark_point(size = 10).encode(
    x ='date:O',
    y ='total_vaccinations:Q',
    color=alt.condition(brush, 'country', alt.ColorValue('gray')),
).add_selection(
    brush
).properties(
    width=350,
    height=350
)

base2 = alt.Chart(df).mark_point(size = 10).encode(
    x ='date:O',
    y ='infection_count:Q',
    color=alt.condition(brush, 'country', alt.ColorValue('gray')),
).add_selection(
    brush
).properties(
    width=350,
    height=350
).interactive()
base1 | base2

alt.HConcatChart(...)

In [11]:
death = pd.read_csv('death_global.csv')
death = death.rename({'Country/Region': 'Country'}, axis=1)
death

,Province/State,Country,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,2683,2686,2698,2710,2713,2721,2730,2733,2742,2745
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,2411,2412,2416,2420,2423,2426,2427,2429,2432,2435
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,3321,3328,3335,3343,3350,3355,3360,3366,3374,3381
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,127,127,127,127,127,127,127,127,127,127
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,630,633,636,639,645,649,651,655,659,677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,NaN,Vietnam,14.058324,108.277199,0,0,0,0,0,0,...,35,35,35,35,35,35,35,36,37,37
271,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,3351,3358,3378,3393,3401,3401,3401,3423,3428,3437
272,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,1270,1271,1275,1276,1277,1278,1285,1285,1289,1294
273,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,1257,1257,1257,1258,1259,1260,1260,1260,1260,1261


In [12]:
data2 = death[death
      .Country
      .isin(['China','India','United Kingdom','Brazil'])].groupby('Country').sum().reset_index().drop(['Lat', 'Long'],axis=1)
df2 = data2.transpose().reset_index()
new_header2 = df2.iloc[0] #grab the first row for the header
df2 = df2[1:]
df2.columns = new_header2
df2 = df2.rename({'Country': 'date'}, axis=1)
df2   

,date,Brazil,China,India,United Kingdom
1,1/22/20,0,17,0,0
2,1/23/20,0,18,0,0
3,1/24/20,0,26,0,0
4,1/25/20,0,42,0,0
5,1/26/20,0,56,0,0
...,...,...,...,...,...
478,5/13/21,430417,4846,262317,127912
479,5/14/21,432628,4846,266207,127930
480,5/15/21,434715,4846,270284,127937
481,5/16/21,435751,4846,274390,127941


In [13]:
death_us = pd.read_csv('death_us.csv')
death_us

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,108,108,108,108,108,108,108,108,108,108
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,308,308,308,308,309,309,309,310,310,310
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,57,57,57,57,56,56,56,56,56,56
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,64,64,64,64,64,64,64,64,64,64
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,137,137,137,137,137,137,139,139,139,139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,...,9,9,9,9,9,9,9,9,9,9
3338,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,...,13,13,13,13,13,13,13,13,13,13
3339,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3340,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,...,26,26,26,26,26,26,26,26,26,26


In [14]:
us_death = death_us.groupby('Country_Region').sum().reset_index().drop(['Lat', 'Long_', 'UID', 'code3', 'FIPS'],axis=1
                                             ).transpose().reset_index()
new_header2 = us_death.iloc[0] #grab the first row for the header
us_death = us_death[1:]
us_death.columns = new_header2
us_death = us_death.rename({'Country_Region':'date', 'US':'United States'}, axis=1)
us_death = us_death.iloc[1: , :]
us_death.index = np.arange(1, len(us_death) + 1)
us_death

,date,United States
1,1/22/20,1
2,1/23/20,1
3,1/24/20,1
4,1/25/20,1
5,1/26/20,1
...,...,...
478,5/13/21,584574
479,5/14/21,585232
480,5/15/21,585708
481,5/16/21,585970


In [15]:
df2 = pd.merge(df2, us_death, how='left', on=['date'])
df2

,date,Brazil,China,India,United Kingdom,United States
0,1/22/20,0,17,0,0,1
1,1/23/20,0,18,0,0,1
2,1/24/20,0,26,0,0,1
3,1/25/20,0,42,0,0,1
4,1/26/20,0,56,0,0,1
...,...,...,...,...,...,...
477,5/13/21,430417,4846,262317,127912,584574
478,5/14/21,432628,4846,266207,127930,585232
479,5/15/21,434715,4846,270284,127937,585708
480,5/16/21,435751,4846,274390,127941,585970


In [16]:
df2['date'] = df2['date'].apply(lambda x : '20' + x[-2:] + '-' + '0'*(2-x.find('/',1)) + x[0:x.find('/',1)] + '-' + '0'*(x.find('/',1)-x.find('/',3)+3) + x[x.find('/',1)+1:x.find('/',3)])
df2

,date,Brazil,China,India,United Kingdom,United States
0,2020-01-22,0,17,0,0,1
1,2020-01-23,0,18,0,0,1
2,2020-01-24,0,26,0,0,1
3,2020-01-25,0,42,0,0,1
4,2020-01-26,0,56,0,0,1
...,...,...,...,...,...,...
477,2021-05-13,430417,4846,262317,127912,584574
478,2021-05-14,432628,4846,266207,127930,585232
479,2021-05-15,434715,4846,270284,127937,585708
480,2021-05-16,435751,4846,274390,127941,585970


In [17]:
cleaned2 = df2.melt('date', var_name='country', value_name='death_count')
new_country = pd.DataFrame(cleaned2.country.unique()).rename(columns={0:'country'})
cleaned2

,date,country,death_count
0,2020-01-22,Brazil,0
1,2020-01-23,Brazil,0
2,2020-01-24,Brazil,0
3,2020-01-25,Brazil,0
4,2020-01-26,Brazil,0
...,...,...,...
2405,2021-05-13,United States,584574
2406,2021-05-14,United States,585232
2407,2021-05-15,United States,585708
2408,2021-05-16,United States,585970


In [18]:
vac = vaccine[vaccine
      .country
      .isin(['China','India','United Kingdom','Brazil','United States'])].reset_index()[['date', 'country', 'total_vaccinations']]
df2 = pd.merge(cleaned2, vac, how='left', on=['date','country']).dropna().reset_index().drop(['index'],axis=1)
df2

,date,country,death_count,total_vaccinations
0,2021-01-16,Brazil,209296,0.0
1,2021-01-17,Brazil,209847,112.0
2,2021-01-18,Brazil,210299,1109.0
3,2021-01-19,Brazil,211491,11470.0
4,2021-01-20,Brazil,212831,28543.0
...,...,...,...,...
537,2021-05-09,United States,581854,259716989.0
538,2021-05-10,United States,582253,261599381.0
539,2021-05-11,United States,582924,263132561.0
540,2021-05-12,United States,583772,264680844.0


## We want to explore the relationship between the number of vaccinations and the nubmer of death across different plots, this time particularly choosing some time slots for comparison.

In [19]:
brush = alt.selection_interval()

base3 = alt.Chart(df2).mark_point(size = 10).encode(
    x ='date:O',
    y ='total_vaccinations:Q',
    color=alt.condition(brush, 'country', alt.ColorValue('gray')),
).add_selection(
    brush
).properties(
    width=350,
    height=350
)

base4 = alt.Chart(df2).mark_point(size = 10).encode(
    x ='date:O',
    y ='death_count:Q',
    color=alt.condition(brush, 'country', alt.ColorValue('gray')),
).add_selection(
    brush
).properties(
    width=350,
    height=350
).interactive()
base3 | base4

alt.HConcatChart(...)